#### Project Title

- Leslie Cohrt: put their contribution here
- Sarah Auther: put their contribution here
- Shoshana Medved: put their contribution here

#### Introduction

blah blah blah blah blah AI becoming more common, free to use, etc, people are developing a relationship with it, idk

- How have people's opinions of ChatGPT evolved as its usage has become more normalized?
- What generates the most reaction between real people when talking about ChatGPT?

Detailed description of dataset

In [4]:
import pandas as pd

df = pd.read_csv("chatgpt.data.csv")
df.head()

,tweet_id,created_at,like_count,quote_count,reply_count,retweet_count,tweet,country,photo_url,city,country_code
0,1.598010e+18,2022-11-30 18:00:15+00:00,2,0,0,0,ChatGPT: Optimizing Language Models for Dialog...,NaN,NaN,NaN,NaN
1,1.598010e+18,2022-11-30 18:02:06+00:00,12179,889,1130,3252,"Try talking with ChatGPT, our new AI system wh...",NaN,NaN,NaN,NaN
2,1.598010e+18,2022-11-30 18:02:58+00:00,2,0,0,1,ChatGPT: Optimizing Language Models for Dialog...,NaN,https://pbs.twimg.com/media/Fi1J8HbWAAMv_yi.jpg,NaN,NaN
3,1.598020e+18,2022-11-30 18:05:58+00:00,561,8,25,66,"THRILLED to share that ChatGPT, our new model ...",NaN,https://pbs.twimg.com/media/Fi1Km3WUYAAfzHS.jpg,NaN,NaN
4,1.598020e+18,2022-11-30 18:06:01+00:00,1,0,0,0,"As of 2 minutes ago, @OpenAI released their ne...",NaN,NaN,NaN,NaN


In [5]:
df.describe()

,tweet_id,like_count,quote_count,reply_count,retweet_count
count,2.192940e+05,219294.000000,219294.000000,219294.000000,219294.000000
mean,1.602573e+18,16.827423,0.403253,1.230412,2.296725
std,3.020433e+15,487.831192,14.095652,25.448931,63.453391
min,1.598010e+18,0.000000,0.000000,0.000000,0.000000
25%,1.600100e+18,0.000000,0.000000,0.000000,0.000000
50%,1.601760e+18,1.000000,0.000000,0.000000,0.000000
75%,1.604880e+18,3.000000,0.000000,1.000000,0.000000
max,1.609340e+18,119321.000000,4598.000000,5184.000000,10593.000000


In [18]:
# preprocessing

import numpy as np

df1 = df.fillna(value=0)

df1.describe()

,tweet_id,like_count,quote_count,reply_count,retweet_count
count,2.192940e+05,219294.000000,219294.000000,219294.000000,219294.000000
mean,1.602573e+18,16.827423,0.403253,1.230412,2.296725
std,3.020433e+15,487.831192,14.095652,25.448931,63.453391
min,1.598010e+18,0.000000,0.000000,0.000000,0.000000
25%,1.600100e+18,0.000000,0.000000,0.000000,0.000000
50%,1.601760e+18,1.000000,0.000000,0.000000,0.000000
75%,1.604880e+18,3.000000,0.000000,1.000000,0.000000
max,1.609340e+18,119321.000000,4598.000000,5184.000000,10593.000000


In [19]:
df1.head()

,tweet_id,created_at,like_count,quote_count,reply_count,retweet_count,tweet,country,photo_url,city,country_code
0,1.598010e+18,2022-11-30 18:00:15+00:00,2,0,0,0,ChatGPT: Optimizing Language Models for Dialog...,0,0,0,0
1,1.598010e+18,2022-11-30 18:02:06+00:00,12179,889,1130,3252,"Try talking with ChatGPT, our new AI system wh...",0,0,0,0
2,1.598010e+18,2022-11-30 18:02:58+00:00,2,0,0,1,ChatGPT: Optimizing Language Models for Dialog...,0,https://pbs.twimg.com/media/Fi1J8HbWAAMv_yi.jpg,0,0
3,1.598020e+18,2022-11-30 18:05:58+00:00,561,8,25,66,"THRILLED to share that ChatGPT, our new model ...",0,https://pbs.twimg.com/media/Fi1Km3WUYAAfzHS.jpg,0,0
4,1.598020e+18,2022-11-30 18:06:01+00:00,1,0,0,0,"As of 2 minutes ago, @OpenAI released their ne...",0,0,0,0


In [20]:
df1.count()

tweet_id         219294
created_at       219294
like_count       219294
quote_count      219294
reply_count      219294
retweet_count    219294
tweet            219294
country          219294
photo_url        219294
city             219294
country_code     219294
dtype: int64

In [21]:
!pip install nltk

import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))




[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
!pip install textblob
from textblob import TextBlob

import re

  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)


In [ ]:
# Word tokenization    
df['w_tokenized'] = df['tweet'].apply(word_tokenize)

print(df['w_tokenized'])


#### How have people's opinions of ChatGPT evolved as its usage has become more normalized?
- sentiment analysis
- dependency parsing

#### What generates the most reaction between real people when talking about ChatGPT?
- sentiment analysis